In [69]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [70]:
from htools import hdir, magics

In [71]:
def differences(obj1, obj2, methods=False, **kwargs):
    """Find the differences between two objects of the same type. This is a 
    way to get more detail beyond whether two objects are equal or not.
    
    Parameters
    -----------
    obj1: any type
        An object.
    obj2: same type as obj1
        An object.
    methods: bool
        If True, include methods in the comparison. If False, only attributes
        will be compared. Note that the output may not be particularly 
        interpretable when using method=True; for instance when comparing two
        strings consisting of different characters, we get a lot of output 
        that looks like this:
        
        {'islower': (<function str.islower()>, <function str.islower()>),
        'isupper': (<function str.isupper()>, <function str.isupper()>),...
        'istitle': (<function str.istitle()>, <function str.istitle()>)}
        
        These attributes all reflect the same difference: if obj1 is 'abc'
        and obj2 is 'def', then 
        'abc' != 'def' and 
        'ABC' != 'DEF' abd 
        'Abc' != 'Def'. 
        
        When method=False, we ignore all of these, such that 
        differences('a', 'b') returns {}. Therefore, it is important to 
        carefully consider what differences you care about identifying.
        
    **kwargs: bool
        Can pass args to hdir to include magics or internals.
    
    Returns
    --------
    dict[str, tuple]: Maps attribute name to a tuple of values, where the 
        first is the corresponding value for obj1 and the second is the 
        corresponding value for obj2.
    """
    attr1, attr2 = hdir(obj1, **kwargs), hdir(obj2, **kwargs)
    assert type(obj1) == type(obj2), 'Objects must be the same type.'
    assert attr1.keys() == attr2.keys(), 'Objects must have same attributes.'
    
    diffs = {}
    if obj1 == obj2:
        return diffs
    for (k1, v1), (k2, v2) in zip(attr1.items(), attr2.items()):
        # Only compare non-callable attributes.
        if not (methods or v1 == 'attribute'):
            continue
            
        # Comparisons work differently for numpy arrays.
        val1, val2 = getattr(obj1, k1), getattr(obj2, k2)
        try:
            equal = (val1 == val2).all()
        except AttributeError:
            equal = val1 == val2
            
        # Store values that are different for obj1 and obj2.
        if not equal:
            diffs[k1] = (val1, val2)
            
    return diffs

In [34]:
differences('a', 'b')

{}

In [51]:
%%talk

s = 'abcDEFghi'
s.casefold() == s.lower()
s.casefold() == s.upper()
s.casefold() == s.title()

True

False

False

In [35]:
differences('a', 'b', True)

{'capitalize': (<function str.capitalize()>, <function str.capitalize()>),
 'casefold': (<function str.casefold()>, <function str.casefold()>),
 'center': (<function str.center(width, fillchar=' ', /)>,
  <function str.center(width, fillchar=' ', /)>),
 'count': (<function str.count>, <function str.count>),
 'encode': (<function str.encode(encoding='utf-8', errors='strict')>,
  <function str.encode(encoding='utf-8', errors='strict')>),
 'endswith': (<function str.endswith>, <function str.endswith>),
 'expandtabs': (<function str.expandtabs(tabsize=8)>,
  <function str.expandtabs(tabsize=8)>),
 'find': (<function str.find>, <function str.find>),
 'format': (<function str.format>, <function str.format>),
 'format_map': (<function str.format_map>, <function str.format_map>),
 'index': (<function str.index>, <function str.index>),
 'isalnum': (<function str.isalnum()>, <function str.isalnum()>),
 'isalpha': (<function str.isalpha()>, <function str.isalpha()>),
 'isascii': (<function str.is

In [36]:
differences(3.4, 3.1)

{'real': (3.4, 3.1)}

In [37]:
differences(3.4, 3.0, True)

{'as_integer_ratio': (<function float.as_integer_ratio()>,
  <function float.as_integer_ratio()>),
 'conjugate': (<function float.conjugate()>, <function float.conjugate()>),
 'hex': (<function float.hex()>, <function float.hex()>),
 'is_integer': (<function float.is_integer()>, <function float.is_integer()>),
 'real': (3.4, 3.0)}

In [42]:
d1 = dict(a=1, b=2, c=3)
d2 = dict(a=2, b=3, c=4)

In [44]:
%%talk

d1 == d2
differences(d1, d2)
differences(d1, d2, True)

False

{}

{'clear': (<function dict.clear>, <function dict.clear>),
 'copy': (<function dict.copy>, <function dict.copy>),
 'get': (<function dict.get(key, default=None, /)>,
  <function dict.get(key, default=None, /)>),
 'items': (<function dict.items>, <function dict.items>),
 'keys': (<function dict.keys>, <function dict.keys>),
 'pop': (<function dict.pop>, <function dict.pop>),
 'popitem': (<function dict.popitem>, <function dict.popitem>),
 'setdefault': (<function dict.setdefault(key, default=None, /)>,
  <function dict.setdefault(key, default=None, /)>),
 'update': (<function dict.update>, <function dict.update>),
 'values': (<function dict.values>, <function dict.values>)}

In [47]:
%%talk

d3 = dict(a=1, b=2, c=3)
d1 is d3
d1 == d3
differences(d1, d3)
differences(d1, d3, True)

False

True

{}

{}

In [55]:
differences(1., 2.)

{'real': (1.0, 2.0)}

## 5/25 update

In [48]:
import numpy as np
import torch

from htools import Args, lmap, hdir

In [49]:
def differences(obj1, obj2, methods=False, **kwargs):
    try:
        if obj1 == obj2:
            return {}
    except ValueError:
        pass

#     assert type(obj1) == type(obj2), 'Objects must be the same type.'
    attr1, attr2 = hdir(obj1, **kwargs), hdir(obj2, **kwargs)
    assert attr1.keys() == attr2.keys(), 'Objects must have same attributes.'

    diffs = {}
    for (k1, v1), (k2, v2) in zip(attr1.items(), attr2.items()):
        # Only compare non-callable attributes.
        if not (methods or v1 == 'attribute'):
            continue

        # Comparisons work differently for numpy arrays.
        val1, val2 = getattr(obj1, k1), getattr(obj2, k2)
        try:
            equal = (val1 == val2).all()
        except AttributeError:
            equal = val1 == val2

        # Store values that are different for obj1 and obj2.
        if not equal:
            diffs[k1] = (val1, val2)

    return diffs

In [50]:
a = np.arange(12)
t = torch.arange(0, 10)
o1 = Args(a=a, t=t)
o1

Args(a=array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11]), t=tensor([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]))

In [51]:
a2 = np.arange(12)
t2 = torch.arange(0, 10)
t2[-1] = 99
o2 = Args(a=a2, t=t2)

In [52]:
differences(o1, o2)

{'t': (tensor([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),
  tensor([ 0,  1,  2,  3,  4,  5,  6,  7,  8, 99]))}

In [59]:
differences(o1, t)

TypeError: eq() received an invalid combination of arguments - got (Args), but expected one of:
 * (Tensor other)
      didn't match because some of the arguments have invalid types: (!Args!)
 * (Number other)
      didn't match because some of the arguments have invalid types: (!Args!)


## kwargs alternative

In [112]:
def kwargs_fallback(self, *args, assign=False, **kwargs):
    res = []
    for arg in args:
        val = kwargs.get(arg) or getattr(self, arg)
        res.append(val)
        if assign: setattr(self, arg, val)
    return res if len(res) > 1 else res[0]

In [113]:
class Foo:
    z = 100
    def __init__(self, a, b=3, c=('a', 'b', 'c')):
        self.a, self.b, self.c = a, b, c
        
    def walk(self, d, **kwargs):
        a, b, c = kwargs_fallback(self, 'a', 'b', 'c', **kwargs)
        print(self.a, a, b, c, d)
        a = kwargs_fallback(self, 'a', assign=True, **kwargs)
        print(a, self.a)
        print('z', self.z)
        z = kwargs_fallback(self, 'z', **kwargs)
        print(z, self.z)
        z = kwargs_fallback(self, 'z', assign=True, **kwargs)
        print(z, self.z)

In [114]:
f = Foo(1)

In [115]:
f.walk(9)

1 1 3 ('a', 'b', 'c') 9
1 1
z 100
100 100
100 100


In [116]:
f.walk(9, b=333)

1 1 333 ('a', 'b', 'c') 9
1 1
z 100
100 100
100 100


In [117]:
f.walk(9, a=11, c=['a', 'bc'])

1 11 3 ['a', 'bc'] 9
11 11
z 100
100 100
100 100


In [118]:
f.walk(44, z=-1)

11 11 3 ('a', 'b', 'c') 44
11 11
z 100
-1 100
-1 -1


In [119]:
Foo.z

100

In [120]:
vars(f)

{'a': 11, 'b': 3, 'c': ('a', 'b', 'c'), 'z': -1}

In [129]:
class Foo:

    def __init__(self, a, b=3, c=('a', 'b', 'c')):
        self.a, self.b, self.c = a, b, c

    def walk(self, d, **kwargs):
        a, c = kwargs_fallback(self, 'a', 'c', **kwargs)
        print(self.a, self.b, self.c)
        print(a, c, end='\n\n')

        b, c = kwargs_fallback(self, 'b', 'c', assign=True, **kwargs)
        print(self.a, self.b, self.c)
        print(b, c)

In [130]:
f = Foo(1)
f.walk(d=0, b=10, c=100)

1 3 ('a', 'b', 'c')
1 100

1 10 100
10 100
